# 라이브러리 임포트 & 경로 지정

1) https://github.com/UB-Mannheim/tesseract/wiki : 여기서 
tesseract-ocr-w64-setup-5.5.0.20241111.exe (64 bit) 클릭 후 다운로드.

2) https://github.com/tesseract-ocr/tessdata_best/blob/main/kor_vert.traineddata
: 여기서 kor.traineddata, kor_vert.traineddata 다운로드 진행

In [12]:
# uv add opencv-python pillow pytesseract matplotlib numpy 해야합니다


import cv2
import pytesseract
from PIL import Image

# Windows 환경에서 설치된 Tesseract 실행 파일 경로
pytesseract.pytesseract.tesseract_cmd = r"C:/Program Files/Tesseract-OCR/tesseract.exe"

# OCR 텍스트 추출 함수

입력 이미지를 전처리 한 후에 텍스트로 변환하는 OCR 기본 함수이다.

In [13]:
def extract_text(img_path: str) -> str:
    """
    손글씨 이미지에서 텍스트 추출
    - return : 추출된 문자열
    """
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
    text = pytesseract.image_to_string(
        thresh, lang="kor+eng"
    )  # OCR 실행, 언어 data에 따라 변경할 수 있다.
    return text.strip()

# OCR 실행

In [15]:
# 테스트 이미지 OCR 실행

result = extract_text("C:/emotion_letter/app/tests/data/sample_letter.jpg")
print("추출된 텍스트:\n", result)

추출된 텍스트:
 To. OC 이게

2 0  -
HY .O0 ㆍ
ㄴㅇ 구야   aba

ve) 9 eek oN ia Sab
apy shige sat CL   Sgt a Ble
gem eat

=

ae
rls

S


# 연필 손글씨 전처리 코드

In [18]:
import cv2
import pytesseract


def extract_text_adaptive(img_path: str) -> str:
    """
    연필 손글씨를 위한 OCR 전처리 + 추출
    """
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Adaptive Thresholding (지역별로 이진화)
    thresh = cv2.adaptiveThreshold(
        gray,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,  # 주변 픽셀 가중치 사용
        cv2.THRESH_BINARY,
        31,  # 블록 크기 (홀수여야 함, 15~51 정도로 조정)
        10,  # 상수(C), 값이 크면 글씨가 더 얇아짐, 작으면 두꺼워짐
    )

    # OCR 실행
    custom_config = r"--oem 3 --psm 6"
    text = pytesseract.image_to_string(thresh, lang="kor+eng", config=custom_config)
    return text.strip(), thresh

In [19]:
text, processed_img = extract_text_adaptive(
    "C:/emotion_letter/app/tests/data/sample_letter.jpg"
)
print("추출된 텍스트:\n", text)

# 결과 이미지 저장해서 눈으로 확인
cv2.imwrite("processed.png", processed_img)

추출된 텍스트:
 0909 : Ub Sak 1 잘 4 내지 7 gz 나 지바 Gale 지 개 ‘ab 내 ae = 때 | :
: 1 : | Be weblal           lols. 3 ee a zh 2 Bet. 13. leak bh Xp |  | 대 —


True

# 노트 선 제거 전처리

In [27]:
import cv2
import numpy as np


def remove_lines(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 15, 10
    )

    # 가로줄 검출
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
    detect_lines = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    # 원본에서 선 제거
    no_lines = cv2.subtract(thresh, detect_lines)
    return no_lines, thresh, detect_lines

# 선 제거 된 이미지 확인

In [28]:
img = cv2.imread("C:/emotion_letter/app/tests/data/sample_letter.jpg")
no_lines, thresh, detect_lines = remove_lines(img)

# 결과 저장
cv2.imwrite("01_thresh.png", thresh)  # 기본 이진화
cv2.imwrite("02_detect_lines.png", detect_lines)  # 검출된 선
cv2.imwrite("03_no_lines.png", no_lines)  # 선 제거 후 이미지

True

# 선 제거 + 해상도 처리

In [23]:
import cv2
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r"C:/Program Files/Tesseract-OCR/tesseract.exe"


def ocr_handwritten(img_path: str) -> str:
    img = cv2.imread(img_path)

    # 노트 줄 제거
    processed = remove_lines(img)

    # 해상도 확대
    resized = cv2.resize(processed, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

    # OCR 실행
    custom_config = r"--oem 3 --psm 6"
    text = pytesseract.image_to_string(resized, lang="kor+eng", config=custom_config)
    return text


print(ocr_handwritten("C:/emotion_letter/app/tests/data/sample_letter.jpg"))

1 To . DO al 개 가 배수 00020 ee
다 Saleh. ab alufa)? BE Ub ali SAAR Se Sp
이이 에트 NEA ee 지키기 느 ia eee 가 그 iS pL Een oh 2021 제; | Ho  .
: : 우르 oe : - 때 : | | - 0 \ 그 : 2: ~ |. : : 7 4 we 은 - 222. ： re] 2 >  7 a : . ： ， < | 씨 ㅇ ! a : : : 기고 pS . 2 x * py ㆍ | 수 7 4 \~ a “e  / : . (il | - 때
ee ' yet 대 SH 얄비이 Be es Pe Az : de 이스 ae a ee oe a . 이이 이
0 이 」 터 | . 오 : ~ : AL 3  | Us | al : = > ; le} : | ： a : | : | - : : 터 oe :  ' _ 나 : 때 피 | 7 더 | | ： 이 7 | ~ : ： ; 이 | | 7 -  -  ： | : _ | | a - - 이니 무이 이 어스시 : |  =
ee ee a'r )n/ 자 a

